# FIPS PUB 180-4 Playground

NOTICE: All non-code text in this notebook is from FIPS PUB 180-4 and is for reference only. All data rights concerning this publication belong to those stated in FIPS PUB 180-4. All code in this notebook was written by Eric Kimsey for personal education purposes and is available under the Unlicense.

License for code:
```
This is free and unencumbered software released into the public domain.

Anyone is free to copy, modify, publish, use, compile, sell, or
distribute this software, either in source code form or as a compiled
binary, for any purpose, commercial or non-commercial, and by any
means.

In jurisdictions that recognize copyright laws, the author or authors
of this software dedicate any and all copyright interest in the
software to the public domain. We make this dedication for the benefit
of the public at large and to the detriment of our heirs and
successors. We intend this dedication to be an overt act of
relinquishment in perpetuity of all present and future rights to this
software under copyright law.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

For more information, please refer to <https://unlicense.org>
```


Title: Secure Hash Standard (SHS)

[Link to Publication](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

## 1. INTRODUCTION

This Standard specifies secure hash algorithms, SHA-1, SHA-224, SHA-256, SHA-384, SHA512, SHA-512/224 and SHA-512/256. All of the algorithms are iterative, one-way hash functions that can process a message to produce a condensed representation called a message digest. These algorithms enable the determination of a message’s integrity: any change to the message will, with a very high probability, result in a different message digest. This property is useful in the generation and verification of digital signatures and message authentication codes, and in the generation of random numbers or bits.

Each algorithm can be described in two stages: preprocessing and hash computation. Preprocessing involves padding a message, parsing the padded message into m-bit blocks, and setting initialization values to be used in the hash computation. The hash computation generates a message schedule from the padded message and uses that schedule, along with functions, constants, and word operations to iteratively generate a series of hash values. The final hash value generated by the hash computation is used to determine the message digest.

The algorithms differ most significantly in the security strengths that are provided for the data being hashed. The security strengths of these hash functions and the system as a whole when each of them is used with other cryptographic algorithms, such as digital signature algorithms and keyed-hash message authentication codes, can be found in \[SP 800-57\] and \[SP 800-107\].

Additionally, the algorithms differ in terms of the size of the blocks and words of data that are
used during hashing or message digest sizes. Figure 1 presents the basic properties of these hash
algorithms.

Algorithm | Message Size(bits) | Block Size(bits) | Word Size(bits) | Message Digest Size(bits)
:---|:--:|:--:|:--:|:--:
SHA-1 | $<2^{64}$ | 512 | 32 | 160
SHA-224 | $<2^{64}$ | 512 | 32 | 224
SHA-256 | $<2^{64}$ | 512 | 32 | 256
SHA-384 | $<2^{128}$ | 1024 | 64 | 384
SHA-512 | $<2^{128}$ | 1024 | 64 | 512
SHA-512/224 | $<2^{128}$ | 1024 | 64 | 224
SHA-512/256 | $<2^{128}$ | 1024 | 64 | 256

**Figure 1: Secure Hash Algorithm Properties**

In [ ]:
class Word():
    """
    Base class for 32-bit and 64-bit words
    """

    def __init__(self, num_bits: int, value: int) -> 'Word':
        """Initialize Word.

        Args:
            num_bits (int): Bit length of the word. Should be either 64 or 32 bits.
            value (int): Value to be held in word. Any bits greater than num_bits are truncated.
        
        Returns:
            Word: Word containing initialization value.
        """

        self.num_bits = num_bits

        if self.num_bits == 32:
            self.value = value & 0xFFFFFFFF
        elif self.num_bits == 64:
            self.value = value & 0xFFFFFFFFFFFFFFFF
        else:
            raise Exception('Incorrect bit length for word. Only 32 or 64-bit words are supported.')
        
    def __add__(self, other: 'Word') -> 'Word':
        """Addition modulo operation for the Word.

        Args:
            other (Word): Word to add with.

        Returns:
            Word: Resulting Word from the addition modulo operation.
        """

        return Word(self.num_bits, (self.value + other.value) % (2**self.num_bits))
        
    def __lshift__(self, shift: int) -> 'Word':
        """Bitwise left shift operation for the word.

        Args:
            shift (int): Number of bits to shift.

        Returns:
            Word: Word shifted left by supplied number of bits. Bits above the word's bit length are truncated.
        """

        if self.num_bits == 32:
            return Word(32, ((self.value << shift) & 0xFFFFFFFF))
        elif self.num_bits == 64:
            return Word(64, ((self.value << shift) & 0xFFFFFFFFFFFFFFFF))

    def __rshift__(self, shift: int) -> 'Word':
        """Bitwise right shift operation for the word.

        Args:
            shift (int): Number of bits to shift.

        Returns:
            Word: Word shifted right by supplied number of bits.
        """
        return Word(self.num_bits, self.value >> shift)
    
    def __and__(self, other: 'Word') -> 'Word':
        """Bitwise AND operation for the Word.

        Args:
            other (Word): Word to AND with.

        Returns:
            Word: Resulting Word from the AND operation.
        """
        return Word(self.num_bits, self.value & other.value)
    
    def __invert__(self) -> 'Word':
        """Bitwise invert operation for the Word.

        Returns:
            Word: Compliment of input Word.
        """
        return Word(self.num_bits, ~self.value)
    
    def __or__(self, other: 'Word') -> 'Word':
        """Bitwise OR operation for the Word.

        Args:
            other (Word): Word to OR with.

        Returns:
            Word: Resulting Word from the OR operation.
        """
        return Word(self.num_bits, self.value | other.value)
    
    def __xor__(self, other: 'Word') -> 'Word':
        """Bitwise XOR operation for the Word.

        Args:
            other (Word): Word to XOR with.

        Returns:
            Word: Resulting Word from the XOR operation.
        """
        return Word(self.num_bits, self.value ^ other.value)
    
    def __str__(self) -> str:
        """String representation of the word.

        Returns:
            str: String representation of the word, as hexadecimal.
        """

        # pad = 8 for 32-bit, 16 for 64-bit
        pad = int(self.num_bits/4)
        return hex(self.value)[2:].zfill(pad).upper()

### 2.2 Algorithm Parameters, Symbols, and Terms

#### 2.2.1 Parameters

The following parameters are used in the secure hash algorithm specifications in this Standard.

Parameter|Description
----|:----
$a, b, c, ..., h$ | Working variables that are the $w$-bit words used in the computation of the hash values, $H^{(i)}$.
$H^{(i)}$ | The $i^{th}$ hash value. $H^{(0)}$ is the *initial* hash value; $H^{(N)}$ is the *final* hash value and is used to determine the message digest.
$H^{(i)}_{j}$ | The $j^{th}$ word of the $i^{th}$ hash value, where $H^{(i)}_{0}$ is the left-most word of hash value $i$.
$K_{t}$ | Constant value to be used for the iteration $t$ of the hash computation.
$k$ | Number of zeroes appended to a message during the padding step.
$\ell$ | Length of the message, M, in bits.
$m$ | Number of bits in a message block, $M^{(i)}$.
$M$ | Message to be hashed.
$M^{(i)}$ | Message block $i$, with a size of $m$ bits.
$M^{(i)}_{j}$ | The $j^{th}$ word of the $i^{th}$ message block, where $M^{(i)}_{0}$ is the left-most word of message block $i$.
$n$ | Number of bits to be rotated or shifted when a word is operated upon.
$N$ | Number of blocks in the padded message.
$T$ | Temporary $w$-bit word used in the hash computation.
$w$ | Number of bits in a word.
$W_{t}$ | The $t^{th}$ $w$-bit word of the message schedule.

#### 2.2.1 Parameters

The following parameters are used in the secure hash algorithm specifications in this Standard.

Parameter|Description
----|:----
$a, b, c, ..., h$ | Working variables that are the $w$-bit words used in the computation of the hash values, $H^{(i)}$.
$H^{(i)}$ | The $i^{th}$ hash value. $H^{(0)}$ is the *initial* hash value; $H^{(N)}$ is the *final* hash value and is used to determine the message digest.
$H^{(i)}_{j}$ | The $j^{th}$ word of the $i^{th}$ hash value, where $H^{(i)}_{0}$ is the left-most word of hash value $i$.
$K_{t}$ | Constant value to be used for the iteration $t$ of the hash computation.
$k$ | Number of zeroes appended to a message during the padding step.
$\ell$ | Length of the message, M, in bits.
$m$ | Number of bits in a message block, $M^{(i)}$.
$M$ | Message to be hashed.
$M^{(i)}$ | Message block $i$, with a size of $m$ bits.
$M^{(i)}_{j}$ | The $j^{th}$ word of the $i^{th}$ message block, where $M^{(i)}_{0}$ is the left-most word of message block $i$.
$n$ | Number of bits to be rotated or shifted when a word is operated upon.
$N$ | Number of blocks in the padded message.
$T$ | Temporary $w$-bit word used in the hash computation.
$w$ | Number of bits in a word.
$W_{t}$ | The $t^{th}$ $w$-bit word of the message schedule.

## 3. NOTATION AND CONVENTIONS

### 3.1 Bit Strings and Integers

The following terminology related to bit strings and integers will be used.

1. A *hex digit* is an element of the set {`0, 1,…, 9, a,…, f`}. A hex digit is the representation of a 4-bit string. For example, the hex digit “`7`” represents the 4-bit string “`0111`”, and the hex digit “`a`" represents the 4-bit string “`1010`”.

2. A word is a $w$-bit string that may be represented as a sequence of hex digits. To convert a word to hex digits, each 4-bit string is converted to its hex digit equivalent, as described in (1) above. For example, the 32-bit string

 `1010 0001 0000 0011 1111 1110 0010 0011`

 can be expressed as “`a103fe23`”, and the 64-bit string

 ```
 1010 0001 0000 0011 1111 1110 0010 0011
 0011 0010 1110 1111 0011 0000 0001 1010
 ```

 can be expressed as “`a103fe2332ef301a`”.

 *Throughout this specification, the “big-endian” convention is used when expressing both 32- and 64-bit words, so that within each word, the most significant bit is stored in the left-most bit position.*

3. An integer may be represented as a word or pair of words. A word representation of the message length,
$\ell$, in bits, is required for the padding techniques of Sec. 5.1. An integer between $0$ and $2^{32}-1$ *inclusive* may be represented as a 32-bit word. The least significant four bits of the integer are represented by the right-most hex digit of the word representation. For example, the integer $291 = 2^{8} + 2^{5} + 2^{1} + 2^{0} = 256 + 32 + 2 + 1$ is represented by the hex word “`00000123`”.

 The same holds true for an integer between $0$ and $2^{64}-1$ *inclusive*, which may be represented as a 64-bit word.

 If $Z$ is an integer, $0 \leq Z < 2^{64}$, then $Z = 2^{32}X + Y$, where $0 \leq X < 2^{32}$ and $0 \leq Y < 2^{32}$. Since $X$ and $Y$ can be represented as 32-bit words $x$ and $y$, respectively, the integer $Z$ can be represented as the pair of words $(x, y)$. This property is used for SHA-1, SHA-224 and SHA-256.

 If $Z$ is an integer, $0 \leq Z < 2^{128}$, then $Z = 2^{64}X + Y$, where $0 \leq X < 2^{64}$ and $0 \leq
Y < 2^{64}$. Since $X$ and $Y$ can be represented as 64-bit words $x$ and $y$, respectively, the integer $Z$
can be represented as the pair of words $(x, y)$. This property is used for SHA-384, SHA-512, SHA-512/224 and SHA-512/256.

4. For the secure hash algorithms, the size of the message block - $m$ bits - depends on the algorithm.
   * For **SHA-1**, **SHA-224** and **SHA-256**, each message block has **512 bits**, which are represented as a sequence of sixteen **32-bit words**.

   * For **SHA-384**, **SHA-512**, **SHA-512/224** and **SHA-512/256** each message block has **1024 bits**, which are represented as a sequence of sixteen **64-bit words**.


### 3.2 Operations on Words

The following operations are applied to w-bit words in all five secure hash algorithms. SHA-1, SHA-224 and SHA-256 operate on 32-bit words ($w=32$), and SHA-384, SHA-512, SHA512/224 and SHA-512/256 operate on 64-bit words ($w=64$).

1. Bitwise logical word operations: $\wedge$, $\vee$, $\oplus$, and $\neg$ (see Sec. 2.2.2).

2. Addition modulo $2^{w}$.

 The operation $x + y$ is defined as follows. The words $x$ and $y$ represent integers $X$ and $Y$, where $0 \leq X < 2^{w}$ and $0 \leq Y < 2^{w}$. For positive integers $U$ and $V$, let $U$ mod$V$ be the remainder upon dividing $U$ by $V$. Compute 

 $Z = (X + Y)$ mod$2^{w}$.

 Then $0 \leq Z < 2^{w}$. Convert the integer $Z$ to a word, $z$, and define $z = x + y$.

3. The right shift operation $SHR^{n}(x)$, where $x$ is a $w$-bit word and $n$ is an integer with $0 \leq n < w$, is defined by 

 $SHR^{n}(x) = x >> n$.

 This operation is used in the SHA-224, SHA-256, SHA-384, SHA-512, SHA-512/224 and SHA-512/256 algorithms.

4. The rotate right (circular right shift) operation $ROTR^{n}(x)$, where $x$ is a $w$-bit word and $n$ is an integer with $0 \leq n < w$, is defined by 

 $ROTR^{n}(x) = (x >> n) \vee (x << w - n)$.

 Thus, $ROTR^{n}(x)$ is equivalent to a circular shift (rotation) of $x$ by $n$ positions to the right.
 
 This operation is used by the SHA-224, SHA-256, SHA-384, SHA-512, SHA512/224 and SHA-512/256 algorithms.

5. The rotate left (circular left shift) operation, $ROTL^{n}(x)$, where $x$ is a $w$-bit word and $n$ is an integer with $0 \leq n < w$, is defined by 

 $ROTL^{n}(x) = (x << n) \vee (x >> w - n)$.

 Thus, ROTL^{n}(x) is equivalent to a circular shift (rotation) of $x$ by $n$ positions to the left.

 This operation is used only in the SHA-1 algorithm.

6. Note the following equivalence relationships, where $w$ is fixed in each relationship:

 $ROTL^{n}(x) \approx ROTR^{w-n}(x)$

 $ROTR^{n}(x) \approx ROTL^{w-n}(x)$


In [ ]:
"""
Word operations
"""
def print_block(w: list[Word]) -> None:
    """Utility function to print an input block of words.

    Args:
        w (list[Word]): List of Word objects, either 32-bit or 64-bit.
    
    Returns:
        None
    """
    for i in range(len(w)):
        print('W[' + str(i) + '] = ' + str(w[i]))
        i = i + 1

def rotate_right(x: Word, n: int) -> Word:
    """Rotate right function for Words.

    Args:
        x (Word): Input Word to rotate.
        n (int): Amount of bits to rotate.

    Returns:
        Word: Word, rotated the number of input bits.
    """
    return (x >> n) | (x << (x.num_bits - n))

def rotate_left(x: Word, n: int) -> Word:
    """Rotate left function for Words.

    Args:
        x (Word): Input Word to rotate.
        n (int): Amount of bits to rotate.

    Returns:
        Word: Word, rotated the number of input bits.
    """
    return ((x << n) & Word(32, 0xFFFFFFFF)) | (x >> (x.num_bits - n))

## 4. FUNCTIONS AND CONSTANTS

### 4.1 Functions

This section defines the functions that are used by each of the algorithms. Although the SHA224, SHA-256, SHA-384,SHA-512, SHA-512/224 and SHA-512/256 algorithms all use similar functions, their descriptions are separated into sections for SHA-224 and SHA-256 (Sec. 4.1.2) and for SHA-384, SHA-512, SHA-512/224 and SHA-512/256 (Sec. 4.1.3), since the input and output for these functions are words of different sizes. Each of the algorithms include $Ch(x, y, z)$ and $Maj(x, y, z)$ functions; the exclusive-OR operation ($\oplus$) in these functions may be replaced by a bitwise OR operation ($\vee$) and produce identical results.

#### 4.1.1 SHA-1 Functions

SHA-1 uses a sequence of logical functions, $f_{0}$, $f_{1}$,..., $f_{79}$. Each function $f_{t}$, where $0 \leq t \leq 79$, operates on three 32-bit words, $x$, $y$, and $z$, and produces a 32-bit word as output. The function $f_{t}(x, y, z)$ is defined as follows:

**Equation 4.1**
$$
f_{t}(x, y, z) =
\begin{cases}
Ch(x, y, z) = (x \wedge y) \oplus ( \neg x \wedge z) &0 \leq t \leq 19 \\
Parity(x, y, z) = x \oplus y \oplus z &20 \leq t \leq 39 \\
Maj(x, y, z) = (x \wedge y) \oplus (x \wedge z) \oplus (y \wedge z) &40 \leq t \leq 59 \\
Parity(x, y, z) = x \oplus y \oplus z &60 \leq t \leq 79.
\end{cases}
$$

In [ ]:
"""
SHA logical functions
"""
def ch(x: Word, y: Word, z: Word) -> Word:
    """Ch function for either 32-bit or 64-bit words.

    Args:
        x (Word): Input word.
        y (Word): Input word.
        z (Word): Input word.

    Returns:
        Word: Output word from Ch function.
    """
    return (x & y) ^ (~x & z)

def maj(x: Word, y: Word, z: Word) -> Word:
    """Maj function for either 32-bit or 64-bit words.

    Args:
        x (Word): Input word.
        y (Word): Input word.
        z (Word): Input word.

    Returns:
        Word: Output word from Maj function.
    """
    return (x & y) ^ (x & z) ^ (y & z)

def parity(x: Word, y: Word, z: Word) -> Word:
    """Parity function for either 32-bit or 64-bit words.

    Args:
        x (Word): Input word.
        y (Word): Input word.
        z (Word): Input word.

    Returns:
        Word: Output word from Parity function.
    """
    return x ^ y ^ z

In [ ]:
"""
SHA-1 logical function sequence
"""
def f(t: int, x: Word, y: Word, z: Word) -> Word:
    """Function sequence for SHA-1 logical functions.
    
    Args:
        t (int): Sequence counter.
        x (Word): Input word.
        y (Word): Input word.
        z (Word): Input word.
    
    Returns:
        Word: Output word from logical function.
    """
    if 0 <= t <= 19:
        return ch(x, y, z)
    elif 10 <= t <= 39:
        return parity(x, y, z)
    elif 40 <= t <= 59:
        return maj(x, y, z)
    elif 60 <= t <= 79:
        return parity(x, y, z)
    else:
        raise Exception('f(t): invalid t input. Should be 0 <= t <= 79. Received value: ' + str(t))

#### 4.1.2 SHA-224 and SHA-256 Functions

SHA-224 and SHA-256 both use six logical functions, where *each function operates on 32-bit words*, which are represented as $x$, $y$, and $z$. The result of each function is a new 32-bit word.

$$
\begin{array}{rcl}
Ch(x, y, z) &= &(x \wedge y) \oplus (\neg x \wedge z) & & & & &(4.2) \\
Maj(x, y, z) &= &(x \wedge y) \oplus (x \wedge z) \oplus (y \wedge z) & & & & &(4.3) \\
\sum^{\{256\}}_{0}(x) &= &ROTR^{2}(x) &\oplus &ROTR^{13}(x) &\oplus &ROTR^{22}(x) &(4.4) \\
\sum^{\{256\}}_{1}(x) &= &ROTR^{6}(x) &\oplus &ROTR^{11}(x) &\oplus &ROTR^{25}(x) &(4.5) \\
\sigma^{\{256\}}_{0}(x) &= &ROTR^{7}(x) &\oplus &ROTR^{18}(x) &\oplus &SHR^{3}(x) &(4.6) \\
\sigma^{\{256\}}_{1}(x) &= &ROTR^{17}(x) &\oplus &ROTR^{19}(x) &\oplus &SHR^{10}(x) &(4.7)
\end{array}
$$

In [ ]:
"""
SHA-224 and SHA-256 Methods
"""
def S_256_0(x: Word) -> Word:
    """Sigma 0 function for SHA-224 and SHA-256.

    Args:
        x (Word): Input 32-bit word.

    Returns:
        Word: Output 32-bit word.
    """
    return rotate_right(x, 2) ^ rotate_right(x, 13) ^ rotate_right(x, 22)

def S_256_1(x: Word) -> Word:
    """Sigma 1 function for SHA-224 and SHA-256.

    Args:
        x (Word): Input 32-bit word.

    Returns:
        Word: Output 32-bit word.
    """
    return rotate_right(x, 6) ^ rotate_right(x, 11) ^ rotate_right(x, 25)

def s_256_0(x: Word) -> Word:
    """sigma 0 function for SHA-224 and SHA-256.

    Args:
        x (Word): Input 32-bit word.

    Returns:
        Word: Output 32-bit word.
    """
    return rotate_right(x, 7) ^ rotate_right(x, 18) ^ (x >> 3)

def s_256_1(x: Word) -> Word:
    """sigma 1 function for SHA-224 and SHA-256.

    Args:
        x (Word): Input 32-bit word.

    Returns:
        Word: Output 32-bit word.
    """
    return rotate_right(x, 17) ^ rotate_right(x, 19) ^ (x >> 10)

#### 4.1.3 SHA-384, SHA-512, SHA-512/224 and SHA-512/256 Functions

SHA-384, SHA-512, SHA-512/224 and SHA-512/256 use six logical functions, where *each function operates on 64-bit words*, which are represented as *x*, *y*, and *z*. The result of each function is a new 64-bit word.

$$
\begin{array}{rcl}
Ch(x, y, z) &= &(x \wedge y) \oplus (\neg x \wedge z) & & & & &(4.8) \\
Maj(x, y, z) &= &(x \wedge y) \oplus (x \wedge z) \oplus (y \wedge z) & & & & &(4.9) \\
\sum^{\{512\}}_{0}(x) &= &ROTR^{28}(x) &\oplus &ROTR^{34}(x) &\oplus &ROTR^{39}(x) &(4.10) \\
\sum^{\{512\}}_{1}(x) &= &ROTR^{14}(x) &\oplus &ROTR^{18}(x) &\oplus &ROTR^{41}(x) &(4.11) \\
\sigma^{\{512\}}_{0}(x) &= &ROTR^{1}(x) &\oplus &ROTR^{8}(x) &\oplus &SHR^{7}(x) &(4.12) \\
\sigma^{\{512\}}_{1}(x) &= &ROTR^{19}(x) &\oplus &ROTR^{61}(x) &\oplus &SHR^{6}(x) &(4.13)
\end{array}
$$


## 4.2 Constants

### 4.2.1 SHA-1 Constants

SHA-1 uses a sequence of eighty constant 32-bit words, $K_{0}$, $K_{1}$,..., $K_{79}$, which are given by

**Equation 4.14**

$$
K_{t}
\begin{cases}
\text{5a827999} &0 \leq t \leq 19 \\
\text{6ed9eba1} &20 \leq t \leq 39 \\
\text{8f1bbcdc} &40 \leq t \leq 59 \\
\text{ca62c1d6} &60 \leq t \leq 79
\end{cases}
$$

In [ ]:
#TODO: Should probably implement the sequence in the main SHA-1 method.

SHA_1_CONSTANTS = [ Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), \
                    Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), \
                    Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), \
                    Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), \
                    Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), Word(32,0x5a827999), \
                    Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), \
                    Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), \
                    Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), \
                    Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), \
                    Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), Word(32,0x6ed9eba1), \
                    Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), \
                    Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), \
                    Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), \
                    Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), \
                    Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), Word(32,0x8f1bbcdc), \
                    Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), \
                    Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), \
                    Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), \
                    Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), \
                    Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6), Word(32,0xca62c1d6)  ]

### 4.2.2 SHA-224 and SHA-256 Constants

SHA-224 and SHA-256 use the same sequence of sixty-four constant 32-bit words, $K^{\{256\}}_{0}$, $K^{\{256\}}_{1}$,...,$K^{\{256\}}_{63}$. These words represent the first thirty-two bits of the fractional parts of the cube roots of the first sixty-four prime numbers. In hex, these constant words are (from left to right):

```
428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5
d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174
e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da
983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967
27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85
a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070
19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3
748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2
```

In [ ]:
SHA_224_256_CONSTANTS = [   Word(32,0x428a2f98), Word(32,0x71374491), Word(32,0xb5c0fbcf), Word(32,0xe9b5dba5), \
                        Word(32,0x3956c25b), Word(32,0x59f111f1), Word(32,0x923f82a4), Word(32,0xab1c5ed5), \
                        Word(32,0xd807aa98), Word(32,0x12835b01), Word(32,0x243185be), Word(32,0x550c7dc3), \
                        Word(32,0x72be5d74), Word(32,0x80deb1fe), Word(32,0x9bdc06a7), Word(32,0xc19bf174), \
                        Word(32,0xe49b69c1), Word(32,0xefbe4786), Word(32,0x0fc19dc6), Word(32,0x240ca1cc), \
                        Word(32,0x2de92c6f), Word(32,0x4a7484aa), Word(32,0x5cb0a9dc), Word(32,0x76f988da), \
                        Word(32,0x983e5152), Word(32,0xa831c66d), Word(32,0xb00327c8), Word(32,0xbf597fc7), \
                        Word(32,0xc6e00bf3), Word(32,0xd5a79147), Word(32,0x06ca6351), Word(32,0x14292967), \
                        Word(32,0x27b70a85), Word(32,0x2e1b2138), Word(32,0x4d2c6dfc), Word(32,0x53380d13), \
                        Word(32,0x650a7354), Word(32,0x766a0abb), Word(32,0x81c2c92e), Word(32,0x92722c85), \
                        Word(32,0xa2bfe8a1), Word(32,0xa81a664b), Word(32,0xc24b8b70), Word(32,0xc76c51a3), \
                        Word(32,0xd192e819), Word(32,0xd6990624), Word(32,0xf40e3585), Word(32,0x106aa070), \
                        Word(32,0x19a4c116), Word(32,0x1e376c08), Word(32,0x2748774c), Word(32,0x34b0bcb5), \
                        Word(32,0x391c0cb3), Word(32,0x4ed8aa4a), Word(32,0x5b9cca4f), Word(32,0x682e6ff3), \
                        Word(32,0x748f82ee), Word(32,0x78a5636f), Word(32,0x84c87814), Word(32,0x8cc70208), \
                        Word(32,0x90befffa), Word(32,0xa4506ceb), Word(32,0xbef9a3f7), Word(32,0xc67178f2)  ]

#### 4.2.3 SHA-384, SHA-512, SHA-512/224 and SHA-512/256 Constants

SHA-384, SHA-512, SHA-512/224 and SHA-512/256 use the same sequence of eighty constant 64-bit words, $K^{\{512\}}_{0}$, $K^{\{512\}}_{1}$,..., $K^{\{512\}}_{79}$. These words represent the first sixty-four bits of the fractional parts of the cube roots of the first eighty prime numbers. In hex, these constant words are (from left to right):

```
428a2f98d728ae22 7137449123ef65cd b5c0fbcfec4d3b2f e9b5dba58189dbbc
3956c25bf348b538 59f111f1b605d019 923f82a4af194f9b ab1c5ed5da6d8118
d807aa98a3030242 12835b0145706fbe 243185be4ee4b28c 550c7dc3d5ffb4e2
72be5d74f27b896f 80deb1fe3b1696b1 9bdc06a725c71235 c19bf174cf692694
e49b69c19ef14ad2 efbe4786384f25e3 0fc19dc68b8cd5b5 240ca1cc77ac9c65
2de92c6f592b0275 4a7484aa6ea6e483 5cb0a9dcbd41fbd4 76f988da831153b5
983e5152ee66dfab a831c66d2db43210 b00327c898fb213f bf597fc7beef0ee4
c6e00bf33da88fc2 d5a79147930aa725 06ca6351e003826f 142929670a0e6e70
27b70a8546d22ffc 2e1b21385c26c926 4d2c6dfc5ac42aed 53380d139d95b3df
650a73548baf63de 766a0abb3c77b2a8 81c2c92e47edaee6 92722c851482353b
a2bfe8a14cf10364 a81a664bbc423001 c24b8b70d0f89791 c76c51a30654be30
d192e819d6ef5218 d69906245565a910 f40e35855771202a 106aa07032bbd1b8
19a4c116b8d2d0c8 1e376c085141ab53 2748774cdf8eeb99 34b0bcb5e19b48a8
391c0cb3c5c95a63 4ed8aa4ae3418acb 5b9cca4f7763e373 682e6ff3d6b2b8a3
748f82ee5defb2fc 78a5636f43172f60 84c87814a1f0ab72 8cc702081a6439ec
90befffa23631e28 a4506cebde82bde9 bef9a3f7b2c67915 c67178f2e372532b
ca273eceea26619c d186b8c721c0c207 eada7dd6cde0eb1e f57d4f7fee6ed178
06f067aa72176fba 0a637dc5a2c898a6 113f9804bef90dae 1b710b35131c471b
28db77f523047d84 32caab7b40c72493 3c9ebe0a15c9bebc 431d67c49c100d4c
4cc5d4becb3e42b6 597f299cfc657e2a 5fcb6fab3ad6faec 6c44198c4a475817
```


## 5. PREPROCESSING

Preprocessing consists of three steps: padding the message, $M$ (Sec. 5.1), parsing the message into message blocks (Sec. 5.2), and setting the initial hash value, $H^{(0)}$ (Sec. 5.3).

### 5.1 Padding the Message

The purpose of this padding is to ensure that the padded message is a multiple of 512 or 1024 bits, depending on the algorithm. Padding can be inserted before hash computation begins on a message, or at any other time during the hash computation prior to processing the block(s) that will contain the padding.

#### 5.1.1 SHA-1, SHA-224, and SHA-256
Suppose that the length of the message, $M$, is $\ell$ bits. Append the bit “1” to the end of the message, followed by $k$ zero bits, where $k$ is the smallest, non-negative solution to the equation $\ell + 1 + k \equiv 448 \text{ mod } 512$. Then append the 64-bit block that is equal to the number $\ell$ expressed using a binary representation. For example, the (8-bit ASCII) message “**abc**” has length $8 \times 3 = 24$, so the message is padded with a one bit, then $448 - (24 + 1) = 423$ zero bits, and then the message length, to become the 512-bit padded message

$$
\begin{array}{rcl}
\underbrace{\text{01100001}}_{\text{"a"}} &\underbrace{\text{01100010}}_{\text{"b"}} &\underbrace{\text{01100011}}_{\text{"c"}} &\text{1} &\overbrace{\text{00...00}}^{\text{423}} &\overbrace{\text{00...} \underbrace{\text{011000}}_{\ell = 24}}^{\text{64}}
\end{array}
$$

The length of the padded message should now be a multiple of 512 bits.

In [ ]:
def sha_32bit_pad_message(mb: bytearray) -> bytearray:
    """Preprocessing for 32-bit algorithms: Pad message to align with block size.

    Args:
        mb (bytearray): The input message.

    Returns:
        bytearray: The input message padded to align with the block size with the message length in the last two 32-bit words.
    """
    l = len(mb) * 8
    mb.append(0b10000000)
    blocks = int(len(mb) / 64)
    if (len(mb) % 64) > 56:
        remaining_pad = 64 - (len(mb) % 64) + 56
    else:
        remaining_pad = ((blocks * 64) + 56) - len(mb)

    while remaining_pad > 0:
        mb.append(0b00000000)
        remaining_pad = remaining_pad - 1

    ell = bytearray((l).to_bytes(8, byteorder='big'))
    mb = mb + ell

    return mb

#### 5.1.2 SHA-384, SHA-512, SHA-512/224 and SHA-512/256

Suppose the length of the message $M$, in bits, is $\ell$ bits. Append the bit “1” to the end of the message, followed by $k$ zero bits, where $k$ is the smallest non-negative solution to the equation $\ell + 1 + k \equiv 896 \text{ mod } 1024$. Then append the 128-bit block that is equal to the number $\ell$ expressed using a binary representation. For example, the (8-bit ASCII) message “**abc**” has length $8 \times 3 = 24$, so the message is padded with a one bit, then $896 - (24 + 1) = 871$ zero bits, and then the message length, to become the 1024-bit padded message

$$
\begin{array}{rcl}
\underbrace{\text{01100001}}_{\text{"a"}} &\underbrace{\text{01100010}}_{\text{"b"}} &\underbrace{\text{01100011}}_{\text{"c"}} &\text{1} &\overbrace{\text{00...00}}^{\text{871}} &\overbrace{\text{00...} \underbrace{\text{011000}}_{\ell = 24}}^{\text{128}}
\end{array}
$$

The length of the padded message should now be a multiple of 1024 bits.

### 5.2 Parsing the Message

The message and its padding must be parsed into $N$ $m$-bit blocks. 

#### 5.2.1 SHA-1, SHA-224 and SHA-256

For SHA-1, SHA-224 and SHA-256, the message and its padding are parsed into $N$ 512-bit blocks, $M^{(1)}$, $M^{(2)}$,..., $M^{(N)}$. Since the 512 bits of the input block may be expressed as sixteen 32-bit words, the first 32 bits of message block $i$ are denoted $M^{(i)}_{0}$, the next 32 bits are $M^{(i)}_{1}$, and so on up to $M^{(i)}_{15}$.

In [ ]:
def sha_32bit_parse_message(mb: bytearray) -> list[Word]:
    """Preprocessing for 32-bit algorithms: Parse Message.

    Args:
        mb (bytearray): The input message, padded to align with the block size.

    Returns:
        list[Word]: The message packed into 32-bit words.
    """
    w = []
    l = len(mb)

    i = 0
    while i < l:
        if (l-i == 3):
            word = Word(32,(mb[i] << 24) | (mb[i+1] << 16) | (mb[i+2] << 8))
        elif (l-i == 2):
            word = Word(32,(mb[i] << 24) | (mb[i+1] << 16))
        elif (l-i == 1):
            word = Word(32,mb[i] << 24)
        else:
            word = Word(32,(mb[i] << 24) | (mb[i+1] << 16) | (mb[i+2] << 8) | mb[i+3])

        w.append(word)
        i = i + 4

    return w

#### 5.2.2 SHA-384, SHA-512, SHA-512/224 and SHA-512/256

For SHA-384, SHA-512, SHA-512/224 and SHA-512/256, the message and its padding are parsed into $N$ 1024-bit blocks, $M^{(1)}$, $M^{(2)}$,..., $M^{(N)}$. Since the 1024 bits of the input block may be expressed as sixteen 64-bit words, the first 64 bits of message block $i$ are denoted $M^{(i)}_{0}$, the next 64 bits are $M^{(i)}_{1}$, and so on up to $M^{(i)}_{15}$. 

### 5.3 Setting the Initial Hash Value ($H^{(0)}$)

Before hash computation begins for each of the secure hash algorithms, the initial hash value, $H^{(0)}$, must be set. The size and number of words in $H^{(0)}$ depends on the message digest size.

#### 5.3.1 SHA-1

For SHA-1, the initial hash value, $H^{(0)}$, shall consist of the following five 32-bit words, in hex:

$$
\begin{array}{rcl}
H^{(0)}_0 &= &\text{67452301} \\
H^{(0)}_1 &= &\text{efcdab89} \\
H^{(0)}_2 &= &\text{98badcfe} \\
H^{(0)}_3 &= &\text{10325476} \\
H^{(0)}_4 &= &\text{c3d2e1f0}
\end{array}
$$

In [ ]:
SHA_1_INITIAL_HASH_VAL = [  Word(32,0x67452301), Word(32,0xefcdab89), Word(32,0x98badcfe), Word(32,0x10325476), Word(32,0xc3d2e1f0)  ]

#### 5.3.2 SHA-224

For SHA-224, the initial hash value, $H^{(0)}$, shall consist of the following eight 32-bit words, in hex:

$$
\begin{array}{rcl}
H^{(0)}_0 &= &\text{c1059ed8} \\
H^{(0)}_1 &= &\text{367cd507} \\
H^{(0)}_2 &= &\text{3070dd17} \\
H^{(0)}_3 &= &\text{f70e5939} \\
H^{(0)}_4 &= &\text{ffc00b31} \\
H^{(0)}_5 &= &\text{68581511} \\
H^{(0)}_6 &= &\text{64f98fa7} \\
H^{(0)}_7 &= &\text{befa4fa4}
\end{array}
$$

In [ ]:
SHA_224_INITIAL_HASH_VAL = [    Word(32,0xc1059ed8), Word(32,0x367cd507), Word(32,0x3070dd17), Word(32,0xf70e5939), \
                                Word(32,0xffc00b31), Word(32,0x68581511), Word(32,0x64f98fa7), Word(32,0xbefa4fa4)  ]

#### 5.3.3 SHA-256

For SHA-256, the initial hash value, $H^{(0)}$, shall consist of the following eight 32-bit words, in hex:

$$
\begin{array}{rcl}
H^{(0)}_0 &= &\text{6a09e667} \\
H^{(0)}_1 &= &\text{bb67ae85} \\
H^{(0)}_2 &= &\text{3c6ef372} \\
H^{(0)}_3 &= &\text{a54ff53a} \\
H^{(0)}_4 &= &\text{510e527f} \\
H^{(0)}_5 &= &\text{9b05688c} \\
H^{(0)}_6 &= &\text{1f83d9ab} \\
H^{(0)}_7 &= &\text{5be0cd19}
\end{array}
$$

These words were obtained by taking the first thirty-two bits of the fractional parts of the square roots of the first eight prime numbers.

In [ ]:
SHA_256_INITIAL_HASH_VAL = [    Word(32,0x6a09e667), Word(32,0xbb67ae85), Word(32,0x3c6ef372), Word(32,0xa54ff53a), \
                                Word(32,0x510e527f), Word(32,0x9b05688c), Word(32,0x1f83d9ab), Word(32,0x5be0cd19)  ]

#### 5.3.4 SHA-384

For SHA-384, the initial hash value, $H^{(0)}$, shall consist of the following eight 64-bit words, in hex:

$$
\begin{array}{rcl}
H^{(0)}_0 &= &\text{cbbb9d5dc1059ed8} \\
H^{(0)}_1 &= &\text{629a292a367cd507} \\
H^{(0)}_2 &= &\text{9159015a3070dd17} \\
H^{(0)}_3 &= &\text{152fecd8f70e5939} \\
H^{(0)}_4 &= &\text{67332667ffc00b31} \\
H^{(0)}_5 &= &\text{8eb44a8768581511} \\
H^{(0)}_6 &= &\text{db0c2e0d64f98fa7} \\
H^{(0)}_7 &= &\text{47b5481dbefa4fa4}
\end{array}
$$

These words were obtained by taking the first sixty-four bits of the fractional parts of the square roots of the ninth through sixteenth prime numbers.

#### 5.3.5 SHA-512

For SHA-512, the initial hash value, $H^{(0)}$, shall consist of the following eight 64-bit words, in hex:

$$
\begin{array}{rcl}
H^{(0)}_0 &= &\text{6a09e667f3bcc908} \\
H^{(0)}_1 &= &\text{bb67ae8584caa73b} \\
H^{(0)}_2 &= &\text{3c6ef372fe94f82b} \\
H^{(0)}_3 &= &\text{a54ff53a5f1d36f1} \\
H^{(0)}_4 &= &\text{510e527fade682d1} \\
H^{(0)}_5 &= &\text{9b05688c2b3e6c1f} \\
H^{(0)}_6 &= &\text{1f83d9abfb41bd6b} \\
H^{(0)}_7 &= &\text{5be0cd19137e2179}
\end{array}
$$

These words were obtained by taking the first sixty-four bits of the fractional parts of the square roots of the first eight prime numbers.

#### 5.3.6 SHA-512/t

> ***TO DO***

##### 5.3.6.1 SHA-512/224

> ***TO DO***

##### 5.3.6.2 SHA-512/256

> ***TO DO***

## 6. SECURE HASH ALGORITHMS

In the following sections, the hash algorithms are not described in ascending order of size. SHA256 is described before SHA-224 because the specification for SHA-224 is identical to SHA256, except that different initial hash values are used, and the final hash value is truncated to 224 bits for SHA-224. The same is true for SHA-512, SHA-384, SHA-512/224 and SHA-512/256, except that the final hash value is truncated to 224 bits for SHA-512/224, 256 bits for SHA512/256 or 384 bits for SHA-384.

For each of the secure hash algorithms, there may exist alternate computation methods that yield identical results; one example is the alternative SHA-1 computation described in Sec. 6.1.3. Such alternate methods may be implemented in conformance to this standard.

### 6.1 SHA-1

SHA-1 may be used to hash a message, *M*, having a length of $\ell$ bits, where $0 \leq \ell \leq 2^{64}$. The algorithm uses 1) a message schedule of eighty 32-bit words, 2) five working variables of 32 bits each, and 3) a hash value of five 32-bit words. The final result of SHA-1 is a 160-bit message digest.

The words of the message schedule are labeled $W_{0}$, $W_{1}$,..., $W_{79}$. The five working variables are labeled ***a***, ***b***, ***c***, ***d***, and ***e***. The words of the hash value are labeled $H^{(i)}_{0}$, $H^{(i)}_{1}$,..., $H^{(i)}_{4}$, which will hold the initial hash value, $H^{(0)}$, replaced by each successive intermediate hash value (after each message block is processed), $H^{(i)}$, and ending with the final hash value, $H^{(N)}$. SHA-1 also uses a single temporary word, *T*.

#### 6.1.1 SHA-1 Preprocessing

1. Set the initial hash value, $H^{(0)}$, as specified in Sec. 5.3.1.
2. The message is padded and parsed as specified in Section 5.

#### 6.1.2 SHA-1 Hash Computation

The SHA-1 hash computation uses functions and constants previously defined in Sec. 4.1.1 and Sec. 4.2.1, respectively. Addition (+) is performed modulo 232.

Each message block, $M^{(1)}$, $M^{(2)}$, ..., $M^{(N)}$, is processed in order, using the following steps:

For i=1 to N:

{

1. Prepare the message schedule, {$W_{t}$}:

$$
W_{t}=
\begin{cases}
M^{(i)}_{t} &0 \leq t \leq 15 \\
ROTL^{1}(W_{t-3} \oplus W_{t-8} \oplus W_{t-14} \oplus W_{t-16}) &16 \leq t \leq 79
\end{cases}
$$

2. Initialize the five working variables, ***a***, ***b***, ***c***, ***d***, and ***e***, with the $(i-1)^{st}$ hash value:

$$
a = H^{(i-1)}_{0} \\
b = H^{(i-1)}_{1} \\
c = H^{(i-1)}_{2} \\
d = H^{(i-1)}_{3} \\
e = H^{(i-1)}_{4}
$$

3. For t=0 to 79:

{

$$
T = ROTL^{5}(a)+f_{t}(b,c,d)+e+K_{t}+W_{t} \\
e = d \\
d = c \\
c = ROTL^{30}(b) \\
b = a \\
a = T
$$

}

4. Compute the $i^{th}$ intermediate hash value $H^{(i)}$:

$$
H^{(i)}_{0} = a + H^{(i-1)}_{0} \\
H^{(i)}_{1} = a + H^{(i-1)}_{1} \\
H^{(i)}_{2} = a + H^{(i-1)}_{2} \\
H^{(i)}_{3} = a + H^{(i-1)}_{3} \\
H^{(i)}_{4} = a + H^{(i-1)}_{4}
$$

}

After repeating steps one through four a total of $N$ times (i.e., after processing $M^{(N)}$), the resulting 160-bit message digest of the message, $M$, is

$$
H^{(N)}_{0} \| H^{(N)}_{1} \| H^{(N)}_{2} \| H^{(N)}_{3} \| H^{(N)}_{4}
$$

In [ ]:
"""
Main SHA-1 function
"""
def sha_1(m: str) -> None:
    """SHA-1 algorithm.

    Args:
        m (str): Input message as a string.
    """
    # Print out input
    print("Input Message: " + m + "\n")
    print("==============================================================\n")

    print("Initial hash value:")
    j = 0
    for init_val in SHA_1_INITIAL_HASH_VAL:
        print("H[" + str(j) + "] = " + str(init_val))
        j = j + 1

    # Convert string input to a bytearray for padding
    mb = bytearray(m, "ascii")

    # Preprocessing - Pad Message
    mb = sha_32bit_pad_message(mb)

    # Preprocessing - Parse Message
    M = sha_32bit_parse_message(mb)

    # Initialize hash variables
    H = []
    for init_val in SHA_1_INITIAL_HASH_VAL:
        H.append(init_val)

    for i in range(int(len(M)/16)):
        print("\n==============================================================\n")

        print("Block contents:")
        print_block(M[i*16:(i*16)+16])

        print("\n          A        B        C        D        E")
        
        # 1. Create the message schedule
        m_sched = []
        for t in range(80):
            if t < 16:
                m_sched.append(M[(i*16)+t])
            else:
                print(t)
                m_sched.append(rotate_left(m_sched[t-3] ^ m_sched[t-8] ^ m_sched[t-14] ^ m_sched[t-16], 1))

        # 2. Set working variables
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]

        # 3. Work through the message schedule
        for t in range(80):
            if(t > 9):
                print("t=" + str(t) + ":", end =" ")
            else:
                print("t= " + str(t) + ":", end =" ")
            t = rotate_left(a, 5) + f(t, b, c, d) + e + SHA_1_CONSTANTS[t] + m_sched[t]
            e = d
            d = c
            c = rotate_left(b, 30)
            b = a
            a = t
            print(a, end = " ")
            print(b, end = " ")
            print(c, end = " ")
            print(d, end = " ")
            print(e, end = " ")
        
        # Compute the intermediate hash value
        print("\nH[0] = " + str(H[0]) + " + " + str(a) + " = ", end = "")
        H[0] = a + H[0]
        print(H[0])
        print("H[1] = " + str(H[1]) + " + " + str(b) + " = ", end = "")
        H[1] = b + H[1]
        print(H[1])
        print("H[2] = " + str(H[2]) + " + " + str(c) + " = ", end = "")
        H[2] = c + H[2]
        print(H[2])
        print("H[3] = " + str(H[3]) + " + " + str(d) + " = ", end = "")
        H[3] = d + H[3]
        print(H[3])
        print("H[4] = " + str(H[4]) + " + " + str(e) + " = ", end = "")
        H[4] = e + H[4]
        print(H[4])

    print("\n--------------------------------------------------------------\n")

    print("Message digest is")
    print("\t" + str(H[0]) + " " + str(H[1]) + " " + str(H[2]) + " " + str(H[3]) + " " + str(H[4]))

#### 6.1.3 Alternate Method for Computing a SHA-1 Message Digest

> ***TO DO***

### 6.2 SHA-256

SHA-256 may be used to hash a message, $M$, having a length of $\ell$ bits, where $0 \leq \ell \leq 2^{64}$. The algorithm uses 1) a message schedule of sixty-four 32-bit words, 2) eight working variables of 32 bits each, and 3) a hash value of eight 32-bit words. The final result of SHA-256 is a 256-bit message digest.

The words of the message schedule are labeled $W_{0}$, $W_{1}$,..., $W_{63}$. The eight working variables are labeled **a**, **b**, **c**, **d**, **e**, **f**, **g**, and **h**. The words of the hash value are labeled $H^{(i)}_{0}$, $H^{(i)}_{1}$,..., $H^{(i)}_{7}$, which will hold the initial hash value, $H^{(0)}$, replaced by each successive intermediate hash value (after each message block is processed), $H^{(i)}$, and ending with the final hash value, $H^{(N)}$. SHA-256 also uses two temporary words, $T_{1}$ and $T_{2}$.

#### 6.2.1 SHA-256 Preprocessing

1. Set the initial hash value, $H^{(0)}$, as specified in Sec. 5.3.3.
2. The message is padded and parsed as specified in Section 5.

#### 6.2.2 SHA-256 Hash Computation

The SHA-256 hash computation uses functions and constants previously defined in Sec. 4.1.2 and Sec. 4.2.2, respectively. Addition ($+$) is performed modulo $2^{32}$.

Each message block, $M^{(1)}$, $M^{(2)}$,..., $M^{(N)}$, is processed in order, using the following steps:

For i=1 to N:

{

1. Prepare the message schedule, $\{W_{t}\}$:

$$
W_{t}=
\begin{cases}
M^{(i)}_{t} &0 \leq t \leq 15 \\
\sigma^{\{256\}}_{1}(W_{t-2})+W_{t-7}+\sigma^{\{256\}}_{0}(W_{t-15})+W_{t-16} &16 \leq t \leq 63
\end{cases}
$$

2. Initialize the eight working variables, **a**, **b**, **c**, **d**, **e**, **f**, **g**, and **h**, with the $(i-1)^{st}$ hash value:

$$
a = H^{(i-1)}_{0} \\
b = H^{(i-1)}_{1} \\
c = H^{(i-1)}_{2} \\
d = H^{(i-1)}_{3} \\
e = H^{(i-1)}_{4} \\
f = H^{(i-1)}_{5} \\
g = H^{(i-1)}_{6} \\
h = H^{(i-1)}_{7}
$$

3. For t=0 to 63:

{

$$
T_{1} = h + \Sigma^{\{256\}}_{1}(e)+Ch(e, f, g)+K^{\{256\}}_{t}+W_{t} \\
T_{2} = \Sigma^{\{256\}}_{0}(a)+Maj(a, b, c) \\
h = g \\
g = f \\
f = e \\
e = d + T_{1} \\
d = c \\
c = b \\
b = a \\
a = T_{1} + T_{2}
$$

}

4. Compute the $i^{th}$ intermediate hash value $H^{(i)}$:

$$
H^{(i)}_{0} = a + H^{(i-1)}_{0} \\
H^{(i)}_{1} = b + H^{(i-1)}_{1} \\
H^{(i)}_{2} = c + H^{(i-1)}_{2} \\
H^{(i)}_{3} = d + H^{(i-1)}_{3} \\
H^{(i)}_{4} = e + H^{(i-1)}_{4} \\
H^{(i)}_{5} = f + H^{(i-1)}_{5} \\
H^{(i)}_{6} = g + H^{(i-1)}_{6} \\
H^{(i)}_{7} = h + H^{(i-1)}_{7}
$$

After repeating steps one through four a total of $N$ times (i.e., after processing $M^{(N)}$), the resulting 256-bit message digest of the message, $M$, is

$$
H^{(N)}_{0}\|H^{(N)}_{1}\|H^{(N)}_{2}\|H^{(N)}_{3}\|H^{(N)}_{4}\|H^{(N)}_{5}\|H^{(N)}_{6}\|H^{(N)}_{7}
$$


In [ ]:
"""
Main SHA-256 function
"""
def sha_256(m):
    """SHA-256 algorithm.

    Args:
        m (string): Input message as a string.
    """
    # Print out input
    print("Input Message: " + m + "\n")
    print("==============================================================\n")

    print("Initial hash value:")
    j = 0
    for init_val in SHA_256_INITIAL_HASH_VAL:
        print("H[" + str(j) + "] = " + str(init_val))
        j = j + 1

    # Convert string input to a bytearray for padding
    mb = bytearray(m, "ascii")

    # Preprocessing - Pad Message
    mb = sha_32bit_pad_message(mb)

    # Preprocessing - Parse Message
    M = sha_32bit_parse_message(mb)

    # Initialize hash variables
    H = []
    for init_val in SHA_256_INITIAL_HASH_VAL:
        H.append(init_val)

    for i in range(int(len(M)/16)):
        print("\n==============================================================\n")

        print("Block contents:")
        print_block(M[i*16:(i*16)+16])

        print("\n          A        B        C        D        E        F        G        H")
        
        # Create the message schedule
        m_sched = []
        for t in range(64):
            if t < 16:
                m_sched.append(M[(i*16)+t])
            else:
                m_sched.append(s_256_1(m_sched[t-2]) + m_sched[t-7] + s_256_0(m_sched[t-15]) + m_sched[t-16])

        # Set working variables
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
        f = H[5]
        g = H[6]
        h = H[7]

        # Work through the message schedule
        for t in range(64):
            if(t > 9):
                print("t=" + str(t) + ":", end =" ")
            else:
                print("t= " + str(t) + ":", end =" ")
            t1 = h + S_256_1(e) + ch(e, f, g) + SHA_224_256_CONSTANTS[t] + m_sched[t]
            t2 = S_256_0(a) + maj(a, b, c)
            h = g
            g = f
            f = e
            e = d + t1
            d = c
            c = b
            b = a
            a = t1 + t2
            print(a, end = " ")
            print(b, end = " ")
            print(c, end = " ")
            print(d, end = " ")
            print(e, end = " ")
            print(f, end = " ")
            print(g, end = " ")
            print(h)
        
        # Compute the intermediate hash value
        print("\nH[0] = " + str(H[0]) + " + " + str(a) + " = ", end = "")
        H[0] = a + H[0]
        print(H[0])
        print("H[1] = " + str(H[1]) + " + " + str(b) + " = ", end = "")
        H[1] = b + H[1]
        print(H[1])
        print("H[2] = " + str(H[2]) + " + " + str(c) + " = ", end = "")
        H[2] = c + H[2]
        print(H[2])
        print("H[3] = " + str(H[3]) + " + " + str(d) + " = ", end = "")
        H[3] = d + H[3]
        print(H[3])
        print("H[4] = " + str(H[4]) + " + " + str(e) + " = ", end = "")
        H[4] = e + H[4]
        print(H[4])
        print("H[5] = " + str(H[5]) + " + " + str(f) + " = ", end = "")
        H[5] = f + H[5]
        print(H[5])
        print("H[6] = " + str(H[6]) + " + " + str(g) + " = ", end = "")
        H[6] = g + H[6]
        print(H[6])
        print("H[7] = " + str(H[7]) + " + " + str(h) + " = ", end = "")
        H[7] = h + H[7]
        print(H[7])

    print("\n--------------------------------------------------------------\n")

    print("Message digest is")
    print("\t" + str(H[0]) + " " + str(H[1]) + " " + str(H[2]) + " " + str(H[3]) + " " + \
                 str(H[4]) + " " + str(H[5]) + " " + str(H[6]) + " " + str(H[7]))

### 6.3 SHA-224

SHA-224 may be used to hash a message, $M$, having a length of $\ell$ bits, where $0 \leq \ell \leq 2^{64}$. The function is defined in the exact same manner as SHA-256 (Section 6.2), with the following two exceptions:

1. The initial hash value, $H^{(0)}$, shall be set as specified in Sec. 5.3.2; and

2. The 224-bit message digest is obtained by truncating the final hash value, $H(N)$, to its left-most 224 bits:

$$
H^{(N)}_{0} \| H^{(N)}_{1} \| H^{(N)}_{2} \| H^{(N)}_{3} \| H^{(N)}_{4} \| H^{(N)}_{5} \| H^{(N)}_{6}
$$

In [ ]:
"""
Main SHA-224 function
"""
def sha_224(m):
    """SHA-224 algorithm.

    Args:
        m (string): Input message as a string.
    """
    # Print out input
    print("Input Message: " + m + "\n")
    print("==============================================================\n")

    print("Initial hash value:")
    j = 0
    for init_val in SHA_224_INITIAL_HASH_VAL:
        print("H[" + str(j) + "] = " + str(init_val))
        j = j + 1

    # Convert string input to a bytearray for padding
    mb = bytearray(m, "ascii")

    # Preprocessing - Pad Message
    mb = sha_32bit_pad_message(mb)

    # Preprocessing - Parse Message
    M = sha_32bit_parse_message(mb)

    # Initialize hash variables
    H = []
    for init_val in SHA_224_INITIAL_HASH_VAL:
        H.append(init_val)

    for i in range(int(len(M)/16)):
        print("\n==============================================================\n")

        print("Block contents:")
        print_block(M[i*16:(i*16)+16])

        print("\n          A        B        C        D        E        F        G        H")
        
        # Create the message schedule
        m_sched = []
        for t in range(64):
            if t < 16:
                m_sched.append(M[(i*16)+t])
            else:
                m_sched.append(s_256_1(m_sched[t-2]) + m_sched[t-7] + s_256_0(m_sched[t-15]) + m_sched[t-16])

        # Set working variables
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
        f = H[5]
        g = H[6]
        h = H[7]

        # Work through the message schedule
        for t in range(64):
            if(t > 9):
                print("t=" + str(t) + ":", end =" ")
            else:
                print("t= " + str(t) + ":", end =" ")
            t1 = h + S_256_1(e) + ch(e, f, g) + SHA_224_256_CONSTANTS[t] + m_sched[t]
            t2 = S_256_0(a) + maj(a, b, c)
            h = g
            g = f
            f = e
            e = d + t1
            d = c
            c = b
            b = a
            a = t1 + t2
            print(a, end = " ")
            print(b, end = " ")
            print(c, end = " ")
            print(d, end = " ")
            print(e, end = " ")
            print(f, end = " ")
            print(g, end = " ")
            print(h)
        
        # Compute the intermediate hash value
        print("\nH[0] = " + str(H[0]) + " + " + str(a) + " = ", end = "")
        H[0] = a + H[0]
        print(H[0])
        print("H[1] = " + str(H[1]) + " + " + str(b) + " = ", end = "")
        H[1] = b + H[1]
        print(H[1])
        print("H[2] = " + str(H[2]) + " + " + str(c) + " = ", end = "")
        H[2] = c + H[2]
        print(H[2])
        print("H[3] = " + str(H[3]) + " + " + str(d) + " = ", end = "")
        H[3] = d + H[3]
        print(H[3])
        print("H[4] = " + str(H[4]) + " + " + str(e) + " = ", end = "")
        H[4] = e + H[4]
        print(H[4])
        print("H[5] = " + str(H[5]) + " + " + str(f) + " = ", end = "")
        H[5] = f + H[5]
        print(H[5])
        print("H[6] = " + str(H[6]) + " + " + str(g) + " = ", end = "")
        H[6] = g + H[6]
        print(H[6])
        print("H[7] = " + str(H[7]) + " + " + str(h) + " = ", end = "")
        H[7] = h + H[7]
        print(H[7])

    print("\n--------------------------------------------------------------\n")

    print("Message digest is")
    print("\t" + str(H[0]) + " " + str(H[1]) + " " + str(H[2]) + " " + str(H[3]) + " " + \
                 str(H[4]) + " " + str(H[5]) + " " + str(H[6]))

### 6.4 SHA-512

> ***TO DO***

#### 6.4.1 SHA-512 Preprocessing

> ***TO DO***

#### 6.4.2 SHA-512 Hash Computation

> ***TO DO***

### 6.5 SHA-384

> ***TO DO***

#### 6.6 SHA-512/224

> ***TO DO***

#### 6.7 SHA-512/256

> ***TO DO***

## 7. TRUNCATION OF A MESSAGE DIGEST

Some application may require a hash function with a message digest length different than those provided by the hash functions in this Standard. In such cases, a truncated message digest may be used, whereby a hash function with a larger message digest length is applied to the data to be hashed, and the resulting message digest is truncated by selecting an appropriate number of the leftmost bits. For guidelines on choosing the length of the truncated message digest and information about its security implications for the cryptographic application that uses it, see SP 800-107 \[SP 800-107\].

## APPENDIX A: Additional Information

### A.1 Security of the Secure Hash Algorithms

The security of the five hash algorithms, SHA-1, SHA-224, SHA-256, SHA-384, SHA-512, SHA-512/224 and SHA-512/256 is discussed in \[SP 800-107\].

### A.2 Implementation Notes

Examples of SHA-1, SHA-224, SHA-256, SHA-384, SHA-512, SHA-512/224 and SHA512/256 are available at http://csrc.nist.gov/groups/ST/toolkit/examples.html.

### A.3 Object Identifiers
Object identifiers (OIDs) for the SHA-1, SHA-224, SHA-256, SHA-384, SHA-512, SHA512/224 and SHA-512/256 algorithms are posted at http://csrc.nist.gov/groups/ST/crypto_apps_infra/csor/algorithms.html.

## APPENDIX B: REFERENCES

Publication | Description
:---|:---
[FIPS 180-3] | NIST, Federal Information Processing Standards Publication 180-3, Secure Hash Standards (SHS), October 2008.
[SP 800-57] | NIST Special Publication (SP) 800-57, Part 1, Recommendation for Key Management: General, (Draft) May 2011.
[SP 800-107] | NIST Special Publication (SP) 800-107, Recommendation for Applications Using Approved Hash Algorithms, (Revised), (Draft) September 2011.

In [ ]:
"""
Input tests
"""
#sha_256('abc')
#sha_256('abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq')
#sha_1('abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq')
sha_224('abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq')